In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import os
from sklearn.model_selection import train_test_split


from tensorflow.keras.preprocessing.image import ImageDataGenerator

import cv2


from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Flatten

from tensorflow.keras.models import model_from_json

In [16]:
print(os.listdir('own_dataset/1d6/train'))

['5', '1', '4', '3', '6', '2']


In [3]:
train_path= ('own_dataset/1d6/train')
test_path= ('own_dataset/1d6/test')

In [4]:
batch_size_train=30  #,batch_size= batch_size_train
batch_size_valid=30
targetsize= 100
datagen=ImageDataGenerator(rotation_range=180,height_shift_range=0.1,rescale=1.0/255.0
                           ,zoom_range=[0.7,1.1],brightness_range=[0.4,1.2])
train_batches= datagen.flow_from_directory(train_path, target_size=(targetsize,targetsize), 
                                                                      classes=['5', '1', '4', '3', '6', '2'],
                                                                      batch_size= batch_size_train)
test_batches= datagen.flow_from_directory(test_path,  target_size=(targetsize,targetsize), 
                                                                     classes=['5', '1', '4', '3', '6', '2'],
                                                                     batch_size= batch_size_train)

Found 4254 images belonging to 6 classes.
Found 834 images belonging to 6 classes.


In [5]:
def plots(ims, figsize=(20,10), rows=1, interp= False, titles= None):
    if type(ims[0]) is np.ndarray:
        ims = np.array(ims).astype(np.uint8)
        if (ims.shape[-1] != 3):
            ims= ims.transpose((0,1,2,3))
    f= plt.figure(figsize=figsize)
    cols= len(ims)//rows if len(ims) %2 == 0 else len(ims)//rows + 1
    for i in range(len(ims)):
        sp = f.add_subplot(rows, cols, i+1)
        sp.axis('Off')
        if titles is not None:
            sp.set_title(titles[i], fontsize=12)
        plt.imshow(ims[i], interpolation=None if interp else 'none')

In [6]:
reduce_lr = tf.keras.callbacks.LearningRateScheduler(lambda x: 1e-3 *0.95**x)

"\nmodel = Sequential()\nmodel.add(Conv2D(52, (3,3), padding='same', activation='relu', input_shape=(targetsize,targetsize, 3)) )\nmodel.add(MaxPooling2D((2, 2)))\n\n\nmodel.add(Conv2D(filters=104, kernel_size=(3,3), padding='SAME', activation='relu'))\nmodel.add(MaxPooling2D((2, 2)))\n\nmodel.add(Conv2D(filters=208, kernel_size=(3,3), padding='SAME', activation='relu'))\nmodel.add(MaxPooling2D((2, 2)))\n\nmodel.add(Conv2D(filters=104, kernel_size=(3,3), padding='SAME', activation='relu'))\nmodel.add(MaxPooling2D((2, 2)))\n\nmodel.add(Conv2D(filters=104, kernel_size=(3,3), padding='SAME', activation='relu'))\nmodel.add(MaxPooling2D((2, 2)))\n\nmodel.add(Flatten())\nmodel.add(Dense(6, activation='softmax'))\nmodel.summary()\n\n\n\n\nEpoch 1/15\n816/816 - 377s - loss: 1.6924 - accuracy: 0.2627 - val_loss: 1.4492 - val_accuracy: 0.4009 - lr: 0.0010\nEpoch 2/15\n816/816 - 374s - loss: 1.3321 - accuracy: 0.4607 - val_loss: 1.1546 - val_accuracy: 0.5193 - lr: 9.0000e-04\nEpoch 3/15\n816/816 

In [8]:
"""

nfilters = [8, 16]
kernel_sizes = [(4,4), (2,2)]


model = Sequential()
# CONV1 (ReLU) > POOL1
model.add(Conv2D(nfilters[0], kernel_sizes[0], # 8 filters, 4x4
                 strides=(1,1),
                 padding='same', 
                 input_shape=(targetsize,targetsize, 3),activation='relu'))
model.add(MaxPooling2D(pool_size=(8,8),
                       strides=(8,8),
                       padding='same'))
# CONV2 (ReLU) > POOL2
model.add(Conv2D(nfilters[1], kernel_sizes[1], # 16 filters, 2x2
                 strides=(1,1),
                 padding='same',activation='relu'))
model.add(MaxPooling2D(pool_size=(4,4),
                       strides=(4,4),
                       padding='same'))

# Fully connected layer with softmax
model.add(Flatten())
model.add(Dense(6, activation='softmax'))
model.summary()
"""

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 100, 100, 8)       392       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 8)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 13, 13, 16)        528       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 4, 4, 16)          0         
_________________________________________________________________
flatten (Flatten)            (None, 256)               0         
_________________________________________________________________
dense (Dense)                (None, 6)                 1542      
Total params: 2,462
Trainable params: 2,462
Non-trainable params: 0
______________________________________________________

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 100, 100, 8)       392       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 8)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 13, 13, 16)        528       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 4, 4, 16)          0         
_________________________________________________________________
flatten (Flatten)            (None, 256)               0         
_________________________________________________________________
dense (Dense)                (None, 6)                 1542      
Total params: 2,462
Trainable params: 2,462
Non-trainable params: 0
______________________________________________________

In [10]:

"""
model.compile(Adam(lr=.0002), loss='categorical_crossentropy', metrics= ['accuracy'])
history= model.fit_generator(train_batches, steps_per_epoch= len(train_batches) , callbacks=[reduce_lr],
                             validation_data=train_batches, validation_steps= len(train_batches), 
                             epochs=500)
"""

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/500
142/142 [==============================] - ETA: 0s - loss: 1.7932 - accuracy: 0.1728

KeyboardInterrupt: 

In [ ]:
#cp_callback=tf.keras.callbacks.ModelCheckpoint(filepath=os.getcwd(),save_weights_only=True,verbose=1)

In [ ]:
#model.save('/home/ordovas/IRONHACK/dice-scores-recognition/model_d6_augmented_c1.h5')

In [ ]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)

In [17]:

model2 = Sequential()
model2.add(Conv2D(32, (3,3), padding='same', activation='relu', input_shape=(targetsize,targetsize, 3)) )
model2.add(MaxPooling2D((2, 2)))

model2.add(Conv2D(filters=64, kernel_size=(3,3), padding='SAME', activation='relu'))
model2.add(MaxPooling2D((2, 2)))


model2.add(Conv2D(filters=128, kernel_size=(3,3), padding='SAME', activation='relu'))
model2.add(MaxPooling2D((2, 2)))

model2.add(Conv2D(filters=256, kernel_size=(3,3), padding='SAME', activation='relu'))
model2.add(MaxPooling2D((2, 2)))

#model2.add(Conv2D(filters=512, kernel_size=(3,3), padding='SAME', activation='relu'))
#model2.add(MaxPooling2D((2, 2)))

model2.add(Flatten())
model2.add(Dense(6, activation='softmax'))
model2.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 100, 100, 32)      896       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 50, 50, 32)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 50, 50, 64)        18496     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 25, 25, 64)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 25, 25, 128)       73856     
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 12, 12, 128)       0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 12, 12, 256)      

In [18]:
model2.compile(Adam(lr=.0002), loss='categorical_crossentropy', metrics= ['accuracy'])
history2= model2.fit_generator(train_batches, steps_per_epoch= len(train_batches) , 
                             validation_data=test_batches, validation_steps= len(test_batches), 
                             epochs=500)

Epoch 1/500


InvalidArgumentError:  logits and labels must be broadcastable: logits_size=[30,10] labels_size=[30,6]
	 [[node categorical_crossentropy/softmax_cross_entropy_with_logits (defined at <ipython-input-18-4172cec10be9>:2) ]] [Op:__inference_train_function_194116]

Function call stack:
train_function


In [13]:
cp_callback=tf.keras.callbacks.ModelCheckpoint(filepath=os.getcwd(),save_weights_only=True,verbose=1)

In [14]:
model2.save('model_1d6_final.h5')

In [15]:
model2_json = model2.to_json()
with open("model_1d6_final.json", "w") as json_file:
    json_file.write(model2_json)

In [ ]:
# https://machinelearningmastery.com/save-load-keras-deep-learning-models/

